In [1]:
import processing
import ds
import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearRegression from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVR from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.18 when using versi

In [2]:
filename_test = '/home/octo/Dropbox'+ '/SPY22Dec.csv'

In [3]:
data=processing.get_csv_pd(filename_test)
data=processing.BA(data)
data=processing.preprocessing_mar(data)
data_km=ml.kalman_ma(data)
data['ckm']=data.Close-data_km
data['km']=data_km
data['rsi']=talib.RSI(np.array(data.Close.astype('float64')))
data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype('float64')),timeperiod=20)
data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=5)
data=data.dropna()
data['ret']=np.log(data.Close/data.Close.shift(300))
data['AB']=np.where(np.log(data.Close/data.Close.shift(300))>0,1,-1)
data=data[7000:25000].dropna()

In [4]:
data=data.dropna()

### outlier

L3p1_UD_prediction_model_saving_draft

In [4]:
filename_outlr = 'outlr.sav'
outlr_loaded = pickle.load(open(filename_outlr, 'rb'))
predictions_outlr = outlr_loaded.predict(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','ckm', 'rsi', 'atr','mom']])
data['outlr']=predictions_outlr
hmm_loaded = joblib.load("markov_hmm.pkl")  
predictions_hmm = hmm_loaded.predict(data[['Volume','spread','ckm', 'rsi','UD']])
data['h_state']=predictions_hmm

In [97]:
pd.crosstab(index=data.h_state,columns=data.NAP) 

NAP,-1,1
h_state,,
0,2520,12223
1,1323,843
2,332,759


In [5]:
data['NAP']=np.where(data.outlr*np.argmin(np.diag(hmm_loaded.transmat_))<1,-1,1)

### ## Jump and UD

In [79]:
data['UD']=np.where(data.Close>data.km*1.00001,1,np.where(data.Close<data.km*0.99999,-1,0))

In [80]:
# class distribution
print(data.groupby('UD').size())

UD
-1    4719
 0    8319
 1    4962
dtype: int64


In [81]:
data['J']=np.where(np.logical_and(data.Close.diff(300)>0.05,data.UD==1),1,
                     np.where(np.logical_and(data.Close.diff(300)<-0.05,data.UD==-1),-1,0))

In [82]:
# class distribution
print(data.groupby('J').size())

J
-1     1272
 0    16125
 1      603
dtype: int64


In [83]:
X=data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','rsi', 'atr', 'mom']]
Y=data.J

In [11]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

In [84]:
data.J.value_counts()

 0    16125
-1     1272
 1      603
Name: J, dtype: int64

## RF

### Oversample for imbalanced data

In [85]:
sm = SMOTE(random_state=12, ratio = 1.0)
X_o,Y_o = sm.fit_sample(X,Y)

In [86]:
SEED=13
rf = RandomForestClassifier(
    n_estimators=8,
    max_features=6,
    random_state=SEED
)

rf.fit(X_o,Y_o)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=8, n_jobs=1,
            oob_score=False, random_state=13, verbose=0, warm_start=False)

In [87]:
filename_rf = 'rf.sav'
pickle.dump(rf, open(filename_rf, 'wb'))

In [16]:
# Kfold validation
import pandas
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

kfold = model_selection.KFold(n_splits=20, random_state=SEED)
results = model_selection.cross_val_score(rf, X_o,Y_o, cv=kfold)
print(results.mean())

0.8859959763726002


## Sell and cover points

In [91]:
data['sc']=np.where(data.Close.diff(30)>0.04,1,np.where(data.Close.diff(30)<-0.04,-1,0))

In [92]:
data.sc.value_counts()

 0    17455
-1      349
 1      196
Name: sc, dtype: int64

In [93]:
X=data[['askPrice', 'askSize', 'bidPrice', 'bidSize','km','spread','rsi', 'atr', 'mom']]
Y=data.sc

In [94]:
sm = SMOTE(random_state=14, ratio = 1.0)
X_o,Y_o = sm.fit_sample(X,Y)

In [95]:
SEED=13
rf = RandomForestClassifier(
    n_estimators=9,
    max_features=6,
    random_state=SEED
)

rf.fit(X_o,Y_o)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=9, n_jobs=1,
            oob_score=False, random_state=13, verbose=0, warm_start=False)

In [96]:
filename_rf = 'rf-sc.sav'
pickle.dump(rf, open(filename_rf, 'wb'))

In [72]:
# Kfold validation
import pandas
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

kfold = model_selection.KFold(n_splits=20, random_state=SEED)
results = model_selection.cross_val_score(rf, X_o,Y_o, cv=kfold)
print(results.mean())

0.9902433909104618


## AdaBoost

In [ ]:
# AdaBoost Classification
import pandas
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 100
kfold = model_selection.KFold(n_splits=20, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())